In [1]:
import mindspore.ops as ops
import numpy as np
import os
import pandas as pd
import json
from PIL import Image
from mindspore import load_checkpoint, load_param_into_net
from mindspore import CheckpointConfig,ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
import mindspore as ms
from mindspore import nn, Tensor
from mindspore import dataset as ds
from mindspore.train import Model, LossMonitor, Callback
from mindspore.nn import Accuracy
from sklearn.utils import shuffle
from mindspore import CheckpointConfig, ModelCheckpoint```

In [2]:
# 定义基础路径
basic_root = 'cassava-leaf-disease-classification'
image_path = os.path.join(basic_root, 'train_images')
csv_path = os.path.join(basic_root, 'train.csv')
json_path = os.path.join(basic_root, 'label_num_to_disease_map.json')

# 读取CSV和JSON文件
image_csv = pd.read_csv(csv_path)
image_json = json.load(open(json_path))
image_json_tolabel = {int(i): j for i, j in image_json.items()}
image_csv['label_name'] = image_csv['label'].map(image_json_tolabel)

# 只处理前3000张图像
image_csv = image_csv.iloc[:2000]


In [4]:
# 数据增强和归一化
transform = ms.dataset.transforms.Compose([
    
    ms.dataset.vision.RandomHorizontalFlip(),
    ms.dataset.vision.RandomVerticalFlip(),
    ms.dataset.vision.RandomRotation(20),
    ms.dataset.vision.ToTensor(),   
    ms.dataset.vision.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:

class CassavaDataset:
    def __init__(self, annotations, img_dir, transform=None, num_classes=5):
        self.annotations = annotations
        self.img_dir = img_dir
        self.transform = transform
        self.num_classes = num_classes

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.annotations.iloc[index, 1]
        # 转换标签为One-hot编码并确保为二维
        label_one_hot = np.zeros((1, self.num_classes), dtype=np.float32)  # 使用二维数组
        label_one_hot[0, label] = 1
        return image, Tensor(label_one_hot, ms.float32)
     
transforms = ms.dataset.transforms.Compose([
    ms.dataset.vision.Resize((256, 256)),
    ms.dataset.vision.RandomHorizontalFlip(),
    ms.dataset.vision.RandomVerticalFlip(),
    ms.dataset.vision.RandomRotation(20),
    

    ms.dataset.vision.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ms.dataset.vision.ToTensor(),
])   

dataset = CassavaDataset(image_csv, image_path, transform=transforms,num_classes=5)


In [6]:
# 创建 MindSpore 数据集
ms_dataset = ds.GeneratorDataset(dataset, ["image", "label"], shuffle=True)

In [7]:
# 获取数据集的实际大小
dataset_size = len(dataset)
print("Dataset size:", dataset_size)

Dataset size: 2000


In [8]:
# 计算训练集和验证集的大小
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size 
train_size

1600

In [9]:
len(dataset)

2000

In [10]:
train_dataset, val_dataset = ms_dataset.split([train_size, val_size])

[WARNING] ME(12048:13668,MainProcess):2024-07-19-19:16:12.121.920 [mindspore\dataset\engine\datasets.py:2480] Dataset is shuffled before split.


In [11]:
print("Train dataset size:", train_dataset.get_dataset_size())
print("Validation dataset size:", val_dataset.get_dataset_size())

Train dataset size: 1600
Validation dataset size: 400


In [13]:
class ResidualBlock(nn.Cell):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, pad_mode='pad')
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, pad_mode='pad')
        self.bn2 = nn.BatchNorm2d(in_channels)

    def construct(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity  # Add the input x to the output
        out = self.relu(out)
        return out

In [14]:

class CNN(nn.Cell):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.SequentialCell([
            nn.Conv2d(3, 32, kernel_size=3, pad_mode='pad', padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            ResidualBlock(32),
            nn.Conv2d(32, 64, kernel_size=3, pad_mode='pad', padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            ResidualBlock(64),
            nn.Conv2d(64, 128, kernel_size=3, pad_mode='pad', padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            ResidualBlock(128)
        ])
        self.fc_layers = nn.SequentialCell([
            nn.Dense(128 * 32 * 32, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Dense(512, 128),
            nn.ReLU(),
            nn.Dense(128, 5)
        ])

    def construct(self, x):
        x = self.conv_layers(x)
        x = ops.Reshape()(x, (-1, 128 * 32 * 32))
        x = self.fc_layers(x)
        return x
    def trainable_params(self):
        return super(CNN, self).trainable_params()



model = CNN()
 
   

In [15]:
# 检查CUDA是否可用并选择设备
device = ms.context.set_context(mode=ms.context.GRAPH_MODE, device_target="CPU")

In [16]:
# 定义模型、损失函数和优化器
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=False)
optimizer = nn.Momentum(params=model.trainable_params(), learning_rate=0.001, momentum=0.01)

loss_monitor = LossMonitor(per_print_times=1)

cassavamodel = Model(model, criterion, optimizer, metrics={"Accuracy": Accuracy()})

In [17]:
cassavamodel.train(5, train_dataset,callbacks=[loss_monitor]) 

epoch: 1 step: 1, loss is 1.8656156063079834
epoch: 1 step: 2, loss is 1.4584769010543823
epoch: 1 step: 3, loss is 1.5712664127349854
epoch: 1 step: 4, loss is 1.9721381664276123
epoch: 1 step: 5, loss is 2.2915191650390625
epoch: 1 step: 6, loss is 0.6684342622756958
epoch: 1 step: 7, loss is 0.5484127998352051
epoch: 1 step: 8, loss is 0.15597717463970184
epoch: 1 step: 9, loss is 6.352414131164551
epoch: 1 step: 10, loss is 0.5682287812232971
epoch: 1 step: 11, loss is 0.41289278864860535
epoch: 1 step: 12, loss is 2.273200035095215
epoch: 1 step: 13, loss is 5.202949523925781
epoch: 1 step: 14, loss is 0.9405927062034607
epoch: 1 step: 15, loss is 0.4710458219051361
epoch: 1 step: 16, loss is 2.7968122959136963
epoch: 1 step: 17, loss is 4.194579601287842
epoch: 1 step: 18, loss is 3.061596155166626
epoch: 1 step: 19, loss is 2.0740466117858887
epoch: 1 step: 20, loss is 3.3191823959350586
epoch: 1 step: 21, loss is 1.9752196073532104
epoch: 1 step: 22, loss is 0.46177881956100464

In [19]:
steps_per_epoch = train_dataset.get_dataset_size()
config = CheckpointConfig(save_checkpoint_steps=steps_per_epoch)
ckpt_callback = ModelCheckpoint(prefix="mnist", directory="./checkpoint", config=config)
loss_callback = LossMonitor(steps_per_epoch)

In [20]:
cassavamodel.fit(10, train_dataset, val_dataset, callbacks=[ckpt_callback, loss_callback])

epoch: 1 step: 1600, loss is 1.9980413913726807
Eval result: epoch 1, metrics: {'Accuracy': 0.6225}
epoch: 2 step: 1600, loss is 0.8570265769958496
Eval result: epoch 2, metrics: {'Accuracy': 0.6425}
epoch: 3 step: 1600, loss is 0.8822121024131775
Eval result: epoch 3, metrics: {'Accuracy': 0.66}
epoch: 4 step: 1600, loss is 1.3794121742248535
Eval result: epoch 4, metrics: {'Accuracy': 0.625}
epoch: 5 step: 1600, loss is 0.14761272072792053
Eval result: epoch 5, metrics: {'Accuracy': 0.6125}
epoch: 6 step: 1600, loss is 0.24511556327342987
Eval result: epoch 6, metrics: {'Accuracy': 0.64}
epoch: 7 step: 1600, loss is 0.13353675603866577
Eval result: epoch 7, metrics: {'Accuracy': 0.6025}
epoch: 8 step: 1600, loss is 0.17282673716545105
Eval result: epoch 8, metrics: {'Accuracy': 0.66}
epoch: 9 step: 1600, loss is 0.4964216649532318
Eval result: epoch 9, metrics: {'Accuracy': 0.61}
epoch: 10 step: 1600, loss is 1.4970145225524902
Eval result: epoch 10, metrics: {'Accuracy': 0.605}


In [29]:


param_dict = load_checkpoint("checkpoint/mnist_1-6_1600.ckpt")  
load_param_into_net(model, param_dict)

([],
 ['global_step',
  'learning_rate',
  'momentum',
  'moments.conv_layers.0.weight',
  'moments.conv_layers.1.gamma',
  'moments.conv_layers.1.beta',
  'moments.conv_layers.4.conv1.weight',
  'moments.conv_layers.4.bn1.gamma',
  'moments.conv_layers.4.bn1.beta',
  'moments.conv_layers.4.conv2.weight',
  'moments.conv_layers.4.bn2.gamma',
  'moments.conv_layers.4.bn2.beta',
  'moments.conv_layers.5.weight',
  'moments.conv_layers.6.gamma',
  'moments.conv_layers.6.beta',
  'moments.conv_layers.9.conv1.weight',
  'moments.conv_layers.9.bn1.gamma',
  'moments.conv_layers.9.bn1.beta',
  'moments.conv_layers.9.conv2.weight',
  'moments.conv_layers.9.bn2.gamma',
  'moments.conv_layers.9.bn2.beta',
  'moments.conv_layers.10.weight',
  'moments.conv_layers.11.gamma',
  'moments.conv_layers.11.beta',
  'moments.conv_layers.14.conv1.weight',
  'moments.conv_layers.14.bn1.gamma',
  'moments.conv_layers.14.bn1.beta',
  'moments.conv_layers.14.conv2.weight',
  'moments.conv_layers.14.bn2.gamma'

In [30]:
acc = cassavamodel.eval(val_dataset)
acc

{'Accuracy': 0.64}

In [31]:
cassavamodel.train(3, train_dataset,callbacks=[loss_monitor])

epoch: 1 step: 1, loss is 1.931178331375122
epoch: 1 step: 2, loss is 0.5072121024131775
epoch: 1 step: 3, loss is 0.1654486209154129
epoch: 1 step: 4, loss is 3.1891872882843018
epoch: 1 step: 5, loss is 1.5751099586486816
epoch: 1 step: 6, loss is 1.63521409034729
epoch: 1 step: 7, loss is 0.7946819067001343
epoch: 1 step: 8, loss is 1.456412434577942
epoch: 1 step: 9, loss is 2.1961214542388916
epoch: 1 step: 10, loss is 1.667017936706543
epoch: 1 step: 11, loss is 0.35358208417892456
epoch: 1 step: 12, loss is 0.18713706731796265
epoch: 1 step: 13, loss is 0.1853591948747635
epoch: 1 step: 14, loss is 1.2681711912155151
epoch: 1 step: 15, loss is 0.8070141077041626
epoch: 1 step: 16, loss is 1.1728487014770508
epoch: 1 step: 17, loss is 1.9993051290512085
epoch: 1 step: 18, loss is 0.5277007222175598
epoch: 1 step: 19, loss is 1.2655935287475586
epoch: 1 step: 20, loss is 0.30447620153427124
epoch: 1 step: 21, loss is 3.6689813137054443
epoch: 1 step: 22, loss is 1.8269691467285156

In [32]:
cassavamodel.eval(val_dataset)

{'Accuracy': 0.5775}

In [42]:
class TestDataset:
    def __init__(self, csv_file, img_dir, transform=None):
        self.csv_file = csv_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, index):
        img_name = self.csv_file.iloc[index, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

In [43]:


test_csv = pd.read_csv('cassava-leaf-disease-classification/sample_submission.csv')
test_image_path = os.path.join(basic_root, 'test_images')
test_dataset = TestDataset(test_csv, test_image_path, transform=transforms)

In [ ]:
ms_test_dataset = ds.GeneratorDataset(test_dataset, ["image"], shuffle=False)
ms_test_dataset = ms_test_dataset.batch(1)

In [46]:
predictions = []
for data in ms_test_dataset.create_dict_iterator(output_numpy=True):
    image = data["image"]
    pred = cassavamodel.predict(Tensor(image))
    predicted_label = np.argmax(pred.asnumpy(), axis=1)
    predictions.append(predicted_label)


In [47]:
test_csv['label'] = predictions

In [48]:
print(test_csv)

         image_id label
0  2216849948.jpg   [3]


In [49]:
test_csv.to_csv('submission.csv', index=False)